In [161]:
# Project baby otter 
# this otter collects promotions from ssg.com!

import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os 
from config import Config
import openpyxl as op 

In [162]:
url = 'https://www.ssg.com/' 
r = requests.get(url)

if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r.status_code)

In [163]:
events = soup.find_all(class_ = 'ssghero_slide_col')
event_ls = [] 
for event in events : 
    try : 
        link = event.find(href=True)["href"]
        if "nevntId" in link : 
            event_ls.append(event)
        # 확실하게 event인 거만 뽑아내기 (프로모션 없는 단순 제품 나열형 기획전 제외 -> 코드가 다름) 
    except : 
        continue

In [164]:
df = pd.DataFrame()
for event in event_ls : 
    
    #기본정보
    titlemain = event.find(class_ = 'ssghero_titmain').text
    titlesub = event.find(class_ = 'ssghero_titsub').text
    link = event.find(href=True)["href"]

    row = pd.DataFrame.from_dict( [{"titlemain" : titlemain
                        ,"titlesub" : titlesub
                        ,"link" : link
                        }])
    df = pd.concat([df, row], ignore_index = True)

In [165]:
df

,titlemain,titlesub,link
0,단 하루 풀무원데이,"최대 1+1, ~40%할인 + 최대 10%적립 중복 가능",https://event.ssg.com/eventDetail.ssg?nevntId=...
1,가을 TPO 스타일링 위크,센스있는 가을 스타일링 ~12% 쿠폰까지,https://event.ssg.com/eventDetail.ssg?nevntId=...
2,먼데이문 CREAM WEEK,"가을 피부, 크림으로 지킬 시간. 1만원 페이백 추첨 혜택까지",https://event.ssg.com/eventDetail.ssg?nevntId=...
3,스마일 펫페어,~20% 장바구니쿠폰 + 서울동물영화제 초대권 추첨,https://event.ssg.com/eventDetail.ssg?nevntId=...
4,10월의 유아동 Little SSG,10% 장바구니쿠폰 + 유아동 쇼핑 하는 날!,https://event.ssg.com/eventDetail.ssg?nevntId=...
5,아동패션 Try On,Best 아동패션 12% 쿠폰 + 영상으로 만나보세요!,https://event.ssg.com/eventDetail.ssg?nevntId=...
6,2022 대한민국 수산대전,행사 대상 수산물 ~50% 할인,https://event.ssg.com/eventDetail.ssg?nevntId=...
7,스마일클럽 월요일마다 장보기 ~10% 적립,"풀무원은 ~40%할인, 1+1 행사까지",https://event.ssg.com/eventDetail.ssg?nevntId=...
8,SSG 스마일일데이트,구교환 & 한소희와 함께하는 스마일클럽 팬미팅,https://event.ssg.com/eventDetail.ssg?nevntId=...
9,프리미엄 아울렛 위크,브랜드별 F/W 대전,https://event.ssg.com/eventDetail.ssg?nevntId=...


In [166]:
#url 타고 들어가기 
sub_url = df.link[10]
r2 = requests.get(sub_url)

if r2.status_code == 200:
    html = r2.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r2.status_code)

In [167]:
# 1. 날짜, 제목 
#프로모션에 따라 구조가 다르므로 try except 처리 
try : 
    evt_title = soup.find(class_ = 'etxt')
    evt_title = evt_title.text
except :
    evt_title = soup.find(class_ = 'cevent_subject_tit')
    evt_title = evt_title.text

try :
    evt_date = soup.find(class_ = 'edays')
    p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
    evt_date = p.findall(evt_date.text)

except :
    evt_date = soup.find(class_ = 'cevent_data_term')
    p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
    evt_date = p.findall(evt_date.text)

In [168]:
print(evt_title)
print(evt_date)

[SSG.LIVE]공동판매구역 감귤편
['2022-10-05', '2022-10-11']


In [169]:
# 2. 대상 샵 
try :
    evt_mall = soup.find(class_ = 'cm_mall_ic ty_text_s').text
except :
    evt_mall = soup.find(class_ = 'cevent_mall_ic cm_mall_ic ty_circle_m').text
evt_mall = evt_mall.splitlines()
evt_mall = [m for m in evt_mall if m != '']

In [170]:
evt_mall

['SSG.COM', '신세계몰', '이마트몰']

In [171]:
evt_text = soup.find_all('span', class_=['evt', 'link_area', 'blind'])

#
for t in evt_text : 
    print(t.text)

SSG.COM
몰 목록 펼치기
SSG.COM
몰 목록 접기
검색
좋아요
MY SSG
주문배송조회
장바구니
SSG.COM
신세계몰
이마트몰
페이스북
트위터
관심상품 취소
관심상품 등록
판매자가입
신세계그룹
고객센터
SSG 페이스북
SSG 인스타그램
QR코드
위해상품차단시스템 운영매장


In [51]:
for i in range(len(evt_text)):
    text_test = [t.text for t in evt_text][i].splitlines()
    print([t for t in text_test if (t != '') & ('바로가기' not in t) and (t != '') & ('쿠폰' in t) and (t != '') & ('%' in t) or (t != '') & ('구매시' in t)])
    #print(text_test)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [172]:
page_text = soup.text.splitlines()

In [128]:
df = pd.DataFrame()
for event in event_ls : 
    
    #기본정보
    titlemain = event.find(class_ = 'ssghero_titmain').text
    titlesub = event.find(class_ = 'ssghero_titsub').text
    link = event.find(href=True)["href"]

    #url 타고 들어가기 
    sub_url = link 
    r2 = requests.get(sub_url)

    if r2.status_code == 200:
        html = r2.text
        soup = BeautifulSoup(html, 'html.parser')

    else : 
        print(r2.status_code)
    
    # 1. 날짜, 제목 
    #프로모션에 따라 구조가 다르므로 try except 처리 
    try : 
        evt_title = soup.find(class_ = 'etxt')
        evt_title = evt_title.text
    except :
        evt_title = soup.find(class_ = 'cevent_subject_tit')
        evt_title = evt_title.text

    try :
        evt_date = soup.find(class_ = 'edays')
        p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
        evt_date = p.findall(evt_date.text)

    except :
        evt_date = soup.find(class_ = 'cevent_data_term')
        p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
        evt_date = p.findall(evt_date.text)

    #시작일, 종료일 구분 
    if len(evt_date) == 2 : 
        evt_start_dt = evt_date[0]
        evt_end_dt = evt_date[1]
    else : 
        evt_start_dt = evt_date[0]
        evt_end_dt = evt_date[0] 

    # 2. 대상 샵 
    try :
        evt_mall = soup.find(class_ = 'cm_mall_ic ty_text_s').text
    except :
        evt_mall = soup.find(class_ = 'cevent_mall_ic cm_mall_ic ty_circle_m').text
    
    evt_mall = evt_mall.splitlines()
    evt_mall = [m for m in evt_mall if m != '']

    if len(evt_mall) < 1


    # 2. 대상 샵 
    #evt_text = soup.find_all('span', class_=['evt', 'link_area', 'blind'])
    evt_text = soup.text.splitlines()
    evt_text = [t for t in evt_text if  
     (('쿠폰' in t) 
    or ('SSGPAY' in t) 
    or ('%' in t) 
    or ('구매시' in t) 
    or ('이벤트' in t) 
    or ('핫딜' in t)  
    or ('타임딜' in t)
    or ('할인' in t)
    or ('할인쿠폰' in t)
    or ('쿠폰할인' in t)
    or ('스마일클럽' in t)
    or ('선착순' in t)
    or ('무료배송' in t)
    or ('청구' in t))
    and ('유의사항' not in t)
    and ('자세히 보기' not in t)
    and ('재발급' not in t)
    and ('불가능' not in t)
    and ('불가' not in t)
    and ('적용되지 않습니다' not in t)
    and ('쿠폰 다운받기' not in t)
    and ('다운로드' not in t)
    and ('가입하러가기' not in t)
    and ('가입하기' not in t)
    and ('가입 하기' not in t)
    and ('보유쿠폰' not in t)
    and ('결제가격' not in t)
    and ('주문금액' not in t)
    and ('비방댓글' not in t)
    and ('조기종료' not in t)
    and ('기준 금액' not in t)
    and ('결제가격' not in t)
    and ('결제 페이지' not in t)
    and ('결제페이지' not in t)
    and ('제세공과금' not in t)]
    # evt_text = [t for t in evt_text if ('쿠폰' or '%' or '구매시' or '이벤트' or '핫딜' or '타임딜' or '할인' 
    # or '할인쿠폰' or '쿠폰할인' or '스마일클럽' or '선착순' or '무료배송' or '청구' in t) 
    # and ('바로가기' not in t)    

    row = pd.DataFrame.from_dict( [{"titlemain" : titlemain
                        ,"titlesub" : titlesub
                        ,"link" : link
                        ,"evt_title" : evt_title
                        ,"evt_start_dt" : evt_start_dt
                        ,"evt_end_dt" : evt_end_dt
                        ,"evt_mall" : evt_mall
                        ,"evt_text" : evt_text}])
    df = pd.concat([df, row], ignore_index = True)

In [97]:
# 이벤트 제목, 데이트는 우선 문제 없는데 원데이 이벤트들이 어떻게 되는지 모르겠음
# evt_date len이 2일 때는 시작일, 종료일이고 1일 때는 시작일 종료일에 각각 넣어주자 
# 샵도 문제없이 작동함 

In [182]:
df = pd.DataFrame()
for event in event_ls : 
    
    #기본정보
    titlemain = event.find(class_ = 'ssghero_titmain').text
    titlesub = event.find(class_ = 'ssghero_titsub').text
    link = event.find(href=True)["href"]

    sub_url = link
    r2 = requests.get(sub_url)

    if r2.status_code == 200:
        html = r2.text
        soup = BeautifulSoup(html, 'html.parser')

    else : 
        print(r2.status_code)
    # title, date 
    try : 
        evt_title = soup.find(class_ = 'etxt')
        evt_title = evt_title.text
    except :
        evt_title = soup.find(class_ = 'cevent_subject_tit')
        evt_title = evt_title.text

    try :
        evt_date = soup.find(class_ = 'edays')
        p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
        evt_date = p.findall(evt_date.text)

    except :
        evt_date = soup.find(class_ = 'cevent_data_term')
        p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
        evt_date = p.findall(evt_date.text)

    if len(evt_date) == 2 :
        evt_start_dt = evt_date[0]
        evt_end_dt = evt_date[1]
    elif len(evt_date) == 1 :
        evt_start_dt = evt_date[0]
        evt_end_dt = evt_date[0]
    

    # evt_mall 
    # 2. 대상 샵 
    try :
        evt_mall = soup.find(class_ = 'cm_mall_ic ty_text_s').text
    except :
        evt_mall = soup.find(class_ = 'cevent_mall_ic cm_mall_ic ty_circle_m').text
    evt_mall = evt_mall.splitlines()
    evt_mall = [m for m in evt_mall if m != '']

    if len(evt_mall) < 1 :
        evt_mall = 'none'
        
    # text 
    evt_text = soup.text.splitlines()
    evt_text = [ t for t in evt_text 
    if (('쿠폰' in t) 
    or ('%' in t)
    or ('SSGMONEY' in t)
    or ('SSG MONEY' in t)
    or ('첫구매' in t)
    or ('구매시' in t)
    or ('이벤트' in t)
    or ('핫딜' in t)
    or ('타임딜' in t)
    or ('할인' in t)
    or ('할인쿠폰' in t)   
    or ('쿠폰할인' in t)
    or ('스마일클럽' in t)
    or ('SSGPAY' in t)
    or ('쓱배송' in t)
    or ('선착순' in t)
    or ('무료배송' in t)
    or ('청구' in t))
    and ('바로가기' not in t)
    and ('주문금액' not in t)
    and ('최소주문금액' not in t)
    and ('최소 주문금액' not in t)
    and ('기준 금액' not in t)
    and ('기준금액' not in t)
    and ('결제 페이지' not in t)
    and ('결제페이지' not in t)
    and ('보유 쿠폰' not in t)
    and ('불가능' not in t)
    and ('불가' not in t)
    and ('제세공과금' not in t)
    and ('다운로드' not in t)   
    and ('다운로드 받기' not in t)
    and ('다운로드받기' not in t)
    and ('다운받기' not in t)
    and ('통합회원' not in t)
    and ('중복적용' not in t)
    and ('중복 적용' not in t)
    and ('중복적용 불가' not in t)
    and ('비방댓글' not in t)
    and ('조기종료' not in t)
    and ('유의사항' not in t)
    and ('재발급' not in t)
    and ('SOLD OUT' not in t)
    and ('Sold out' not in t)
    and ('쿠폰이 다운되었습니다' not in t)
    and ('고객님은 쿠폰을 이미 다운 받으셨습니다' not in t)
    and ('금일 쿠폰이 소진되었습니다' not in t)
    and ('선착순 사은품 증정이 마감되었습니다' not in t)
    and ('선착순 사은품 증정이 마감되었습니다' not in t)

    ]
    evt_desc = []
    for t in evt_text :
        if t not in evt_desc :
            evt_desc.append(t)

    row = pd.DataFrame.from_dict( [{"titlemain" : titlemain
                        ,"titlesub" : titlesub
                        ,"link" : link
                       ,"evt_title" : evt_title
                       ,"evt_start_dt" : evt_start_dt
                       ,"evt_end_dt" :  evt_end_dt
                       ,"evt_mall" : evt_mall
                       ,"evt_desc" : evt_desc}])
    df = pd.concat([df, row], ignore_index = True)

In [183]:
df

,titlemain,titlesub,link,evt_title,evt_start_dt,evt_end_dt,evt_mall,evt_desc
0,단 하루 풀무원데이,"최대 1+1, ~40%할인 + 최대 10%적립 중복 가능",https://event.ssg.com/eventDetail.ssg?nevntId=...,(10/10) 풀무원 데이,2022-10-10,2022-10-10,"[SSG.COM, 이마트몰, 새벽배송]","[이벤트/쿠폰 > (10/10) 풀무원 데이, 할인도 적립도 중복 가능한 단 하루 ..."
1,가을 TPO 스타일링 위크,센스있는 가을 스타일링 ~12% 쿠폰까지,https://event.ssg.com/eventDetail.ssg?nevntId=...,가을 TPO 스타일링 위크,2022-10-10,2022-10-16,"[SSG.COM, 신세계몰]","[이벤트/쿠폰 > 가을 TPO 스타일링 위크, 패션 쿠폰, 10% 혜택, 스마일클럽..."
2,먼데이문 CREAM WEEK,"가을 피부, 크림으로 지킬 시간. 1만원 페이백 추첨 혜택까지",https://event.ssg.com/eventDetail.ssg?nevntId=...,먼데이문 CREAM WEEK,2022-10-10,2022-10-16,"[SSG.COM, 신세계몰, 신세계백화점]","[이벤트/쿠폰 > 먼데이문 CREAM WEEK, 구매사은 페이백 이벤트, (재입고)..."
3,스마일 펫페어,~20% 장바구니쿠폰 + 서울동물영화제 초대권 추첨,https://event.ssg.com/eventDetail.ssg?nevntId=...,서울동물영화제와 함께하는 스마일 펫페어,2022-10-10,2022-10-16,"[SSG.COM, 신세계몰, 이마트몰, 새벽배송]","[이벤트/쿠폰 > 서울동물영화제와 함께하는 스마일 펫페어, 3종 쿠폰, 초대권 추첨..."
4,10월의 유아동 Little SSG,10% 장바구니쿠폰 + 유아동 쇼핑 하는 날!,https://event.ssg.com/eventDetail.ssg?nevntId=...,[10/10-16] 10월의 Little SSG 유아동 혜택을 만나보세요,2022-10-10,2022-10-16,"[SSG.COM, 신세계몰, 신세계백화점, 이마트몰]",[이벤트/쿠폰 > [10/10-16] 10월의 Little SSG 유아동 혜택을 만...
5,아동패션 Try On,Best 아동패션 12% 쿠폰 + 영상으로 만나보세요!,https://event.ssg.com/eventDetail.ssg?nevntId=...,"[10/10-16] BEST 아동패션 브랜드, 영상으로 생생하게 Try On",2022-10-10,2022-10-16,"[SSG.COM, 신세계몰, 신세계백화점, 이마트몰]","[이벤트/쿠폰 > [10/10-16] BEST 아동패션 브랜드, 영상으로 생생하게 ..."
6,2022 대한민국 수산대전,행사 대상 수산물 ~50% 할인,https://event.ssg.com/eventDetail.ssg?nevntId=...,(10/7~19) 2022 대한민국 수산대전 - 10월 가을맞이 특별전,2022-10-07,2022-10-19,"[SSG.COM, 이마트몰, 트레이더스, 새벽배송]",[이벤트/쿠폰 > (10/7~19) 2022 대한민국 수산대전 - 10월 가을맞이 ...
7,스마일클럽 월요일마다 장보기 ~10% 적립,"풀무원은 ~40%할인, 1+1 행사까지",https://event.ssg.com/eventDetail.ssg?nevntId=...,[스마일클럽] 10월 내내 매일매일 스마일,2022-10-03,2022-10-31,"[SSG.COM, 신세계몰, 이마트몰, 트레이더스, 새벽배송]","[이벤트/쿠폰 > [스마일클럽] 10월 내내 매일매일 스마일, [스마일클럽] 10월..."
8,SSG 스마일일데이트,구교환 & 한소희와 함께하는 스마일클럽 팬미팅,https://event.ssg.com/eventDetail.ssg?nevntId=...,구교환 & 한소희와 함께하는 SSG 스마일일데이트,2022-10-10,2022-10-20,[SSG.COM],"[이벤트/쿠폰 > 구교환 & 한소희와 함께하는 SSG 스마일일데이트, 구교환 & 한..."
9,프리미엄 아울렛 위크,브랜드별 F/W 대전,https://event.ssg.com/eventDetail.ssg?nevntId=...,[1010-1016] 프리미엄 아울렛 위크,2022-10-10,2022-10-16,"[SSG.COM, 신세계몰, 프리미엄아울렛]","[이벤트/쿠폰 > [1010-1016] 프리미엄 아울렛 위크, 최대 10% 할인 쿠..."


In [138]:
df.to_csv('/Users/jh/Desktop/test.csv', encoding = 'utf-8-sig')

In [188]:
df.to_excel('./tmp/tmp.xlsx', encoding = 'utf-8-sig', index = False)

In [185]:
# tmp 파일에 df 저장하고 
# csv 

In [189]:
filepath = './tmp/tmp.xlsx'

wb = op.load_workbook(filepath)
ws = wb.active

In [187]:
for r in ws.rows : 
    #print(r)
    row_index = r[0].row
    #print(row_index)
    if row_index == 1 : 
        continue
    else : 
        ws['I'+str(row_index)] = f'''= SUBSTITUTE(H{row_index}, "', '", CHAR(10)) '''
    
wb.save(filepath)